# Import de tous les modules

In [2]:
import pandas as pd
import numpy as np

On lit le fichier `fr.openfoodfacts.org.products.csv` avec le séparateur `\t`. Le résultat sera dans la variable `data`

In [3]:
chunksize = 10 ** 3
data = None
for chunk in pd.read_csv("fr.openfoodfacts.org.products.csv", chunksize=chunksize, sep="\t"):
    data = chunk
    break

### On regarde le tableau des valeurs qui ne sont pas définies

On regardera surtout les colonnes qui ont plus de 90 % de valeures non définies. Cela nous aidera à savoir s'il est utile de prendre une colonne ou non.

In [4]:
# On lis les valeurs non definies
missing = pd.read_csv("fr.openfoodfacts.org.products.csv", sep="\t", low_memory=False).isna()
# On les compte et on créer un nouveau DataFrame
missing_sum = pd.DataFrame(missing.sum(), columns=['nb_manquants'])
# On calcul et ajoute le ratio au DataFrame
missing_sum["ratio"] = ((missing_sum["nb_manquants"] / len(missing)) )
# On garde les ratio > 0.9
missing_sum_90 = missing_sum[missing_sum["ratio"] > 0.9]
# On garde les ratio == 1
missing_sum_90_1 = missing_sum_90[missing_sum_90["ratio"] == 1]

print(f"Sur {len(missing_sum)} colonnes, il y en a {len(missing_sum_90)} qui ont au moins 90% de valeurs manquantes dont {len(missing_sum_90_1)} qui n'ont aucune valeurs")
print(f"Les colonnes qui n'ont aucune valeurs sont les suivantes :")
print(missing_sum_90_1.index.to_numpy())
# On exporte ces données en .csv pour analyser si besoin
missing_sum.to_csv("valeures_manquantes.csv")
missing_sum_90.to_csv("valeures_manquantes_90.csv")

Sur 162 colonnes, il y en a 102 qui ont au moins 90% de valeurs manquantes dont 17 qui n'ont aucune valeurs
Les colonnes qui n'ont aucune valeurs sont les suivantes :
['no_nutriments' 'ingredients_from_palm_oil'
 'ingredients_that_may_be_from_palm_oil' 'nutrition_grade_uk'
 'butyric-acid_100g' 'caproic-acid_100g' 'lignoceric-acid_100g'
 'cerotic-acid_100g' 'montanic-acid_100g' 'melissic-acid_100g'
 'elaidic-acid_100g' 'mead-acid_100g' 'erucic-acid_100g'
 'nervonic-acid_100g' 'chlorophyl_100g' 'glycemic-index_100g'
 'water-hardness_100g']


# Selection des données qui seront intéressantes pour le projet

Cela permet surtout de ne pas trop utiliser la mémoire pour les données qui ne me seront pas utiles. On gardera les `tags` pour faire nos analyses.
On peut également dors et déjà éliminer les 17 colonnes qui n'ont aucune valeur. Il n'y a donc pas les éléments suivant dans la liste :

`no_nutriments` ; `ingredients_from_palm_oil` ; `ingredients_that_may_be_from_palm_oil` ; `nutrition_grade_uk` ; `butyric-acid_100g` ; `caproic-acid_100g` ; `lignoceric-acid_100g` ; `cerotic-acid_100g` ; `montanic-acid_100g` ; `melissic-acid_100g` ; `elaidic-acid_100g` ; `mead-acid_100g` ; `erucic-acid_100g` ; `nervonic-acid_100g` ; `chlorophyl_100g` ; `glycemic-index_100g` ; `water-hardness_100g`

- ❌ `code`, pour nos analyses on pourra s'en passer
- ✅ `url`, utile pour retrouver la source sur le site
- ✅ `creator`, pas très utile de savoir qui a contribué pour cette donnée
- ✅ `created_t`, date de création peut être utile par exemple pour comparer avec le nombre de modifications.
- ✅ `created_datetime`, même donnée que created_t au format iso8601. On préférera celle-ci à la created_t. On essayera, si une ligne est NaN de voir si la donnée existe dans created_t.
- ✅ `last_modified_t`, dernière modification de la donnée. Peut être utile pour voir si un produit est controversé ou évolue vite.
- ✅ `last_modified_datetime`, pareil que pour created_datetime.
- ✅ `product_name`, utile pour afficher le nom du produit.
- ✅ `generic_name`, utile si le product_name n'est pas exploitable
- ✅ `quantity`, très utile pour de multiple raisons. On peut comparer à beaucoup d'autres attributs
- ✅ `packaging`, utile si besoin pour comparer écologiquement
- ✅ `packaging_tags`, on gardera `packaging_tags` car parfois la nomenclature n'est pas identique pour 2 valeurs qui sont sensé l'être. Par exemple il y a les valeurs "boîte" et "Boîte".
- ✅ `brands`, utile si besoin de connaître la marque qui fabrique le produit
- ✅ `brands_tags`, /
- ✅ `categories`, une possible utilité.
- ✅ `categories_tags`, /
- ✅ `categories_fr`, cette colonne représente le français de `categories`, or dans `categories` il y a du français / anglais mélangé.
- ✅ `origins`, connaître la provenance des ingrédients est utile par exemple pour effectuer un classement de pays par nutri-scrore
- ✅ `origins_tags`, /
- ✅ `manufacturing_places`, intéressant d'avoir le lieux de fabrication du produit.
- ✅ `manufacturing_places_tags`, /
- ✅ `labels`, labels peut être intéressant dans certains cas
- ✅ `labels_tags`, /
- ✅ `labels_fr`, peu de sens puisqu'il y a du français dans `labels` et de l'anglais dans `labels_fr`.
- ❌ `emb_codes`, code interne peu utile. Les tags sont quasiement tous différents
- ❌ `emb_codes_tags`, /
- ❌ `first_packaging_code_geo`, pas assez de données pour être exploité (93% na)
- ❌ `cities`, quasi 100% des données na. Pas utile
- ❌ `cities_tags`, /
- ✅ `purchase_places`, utile pour connaître les lieux de distribution du produit.
- ✅ `stores`, utile pour connaître les distributeurs de ce produit.
- ❌ `countries`, l'origine du produit pour connaître les pays qui exportent le plus de produits différents par exemple. On gardera les _tags et le _fr uniquement.
- ✅ `countries_tags`, /
- ✅ `countries_fr`, /
- ✅ `ingredients_text`,
- ✅ `traces`,
- ✅ `traces_tags`,
- ✅ `serving_size`,
- ✅ `additives_n`,
- ✅ `additives`,
- ✅ `additives_tags`,
- ❌ `ingredients_from_palm_oil_n`, peu de données, majoritairement des 0 pour les valeurs définies
- ❌ `ingredients_from_palm_oil_tags`, peu de données
- ❌ `ingredients_that_may_be_from_palm_oil_n`, certaines données sont insérées mais pas comparable aux autre colonnes...
- ❌ `ingredients_that_may_be_from_palm_oil`, donnée peu intéressante pour mon sujet
- ❌ `ingredients_that_may_be_from_palm_oil_tags`, /
- ✅ `nutrition_grade_fr`,
- ✅ `main_category`,
- ✅ `main_category_fr`,
- ✅ `image_url`,
- ✅ `image_small_url`,
- ✅ `energy_100g`,
energy-kj_100g
energy-kcal_100g
proteins_100g
casein_100g
serum-proteins_100g
nucleotides_100g
carbohydrates_100g
sugars_100g
sucrose_100g
glucose_100g
fructose_100g
lactose_100g
maltose_100g
maltodextrins_100g
starch_100g
polyols_100g
fat_100g
saturated-fat_100g
caprylic-acid_100g
capric-acid_100g
lauric-acid_100g
myristic-acid_100g
palmitic-acid_100g
stearic-acid_100g
arachidic-acid_100g
behenic-acid_100g
monounsaturated-fat_100g
polyunsaturated-fat_100g
omega-3-fat_100g
alpha-linolenic-acid_100g
eicosapentaenoic-acid_100g
docosahexaenoic-acid_100g
omega-6-fat_100g
linoleic-acid_100g
arachidonic-acid_100g
gamma-linolenic-acid_100g
dihomo-gamma-linolenic-acid_100g
omega-9-fat_100g
oleic-acid_100g
gondoic-acid_100g
trans-fat_100g
cholesterol_100g
fiber_100g
sodium_100g
alcohol_100g
vitamin-a_100g
vitamin-d_100g
vitamin-e_100g
vitamin-k_100g
vitamin-c_100g
vitamin-b1_100g
vitamin-b2_100g
vitamin-pp_100g
vitamin-b6_100g
vitamin-b9_100g
vitamin-b12_100g
biotin_100g
pantothenic-acid_100g
silica_100g
bicarbonate_100g
potassium_100g
chloride_100g
calcium_100g
phosphorus_100g
iron_100g
magnesium_100g
zinc_100g
copper_100g
manganese_100g
fluoride_100g
selenium_100g
chromium_100g
molybdenum_100g
iodine_100g
caffeine_100g
taurine_100g
ph_100g
fruits-vegetables-nuts_100g
carbon-footprint_100g : carbon footprint (as indicated on the packaging of some products)
nutrition-score-fr_100g : Nutri-Score - Nutrition score derived from the UK FSA score and adapted for the French market (formula defined by the team of Professor Hercberg)
nutrition-score-uk_100g : nutrition score defined by the UK Food Standards Administration (FSA)

In [ ]:
data = data.filter([
    "url", 
    "created_t", 
    "created_datetime", 
    "last_modified_t",
    "last_modified_datetime", 
    "product_name",
    "generic_name",
    "quantity", 
    "packaging",
    "packaging_tags",
    "brands",
    "categories"
    ], axis=1)

# Vérification des erreurs sur les variables générales

In [ ]:
# Code
